In [5]:
#Fist import numpy library
import numpy as np

In [182]:
#Let's define some useful functions
#Function distance: to compute the distance between the actual location and the goal
# and choosing in which direction to move (the one closest to the goal)
def distance(i,j,cell, rotations):
  r=len(cell)
  c=len(cell[0])
  if rotations%2==0:
    dist1=abs(r-i-3)+abs(c-1-j)
    dist2=abs(r-i-2)+abs(c-j-2)
  else:
    dist1=abs(r-i-2)+abs(c-2-j)
    dist2=abs(r-1-i)+abs(c-j-3)
  if dist1<dist2:
    x=i+1
    y=j
  else:
    x=i
    y=j+1
  return x,y

#Function nearest: to find where to move next looking for the shortest path, choosing
# cells with lower value and, in case both directions have the same value, choosing by the
# distance between each cell and the goal.
def nearest(i,j,cell, rotations):
  near=np.array([])
  if (cell[i+1][j]>cell[i][j+1]):
    near=np.r_[near,[i,j+1]]
  elif (cell[i+1][j]<cell[i][j+1]):
    near=np.r_[near,[i+1,j]]
  elif (cell[i+1][j]==cell[i][j+1]) and (cell[i+1][j]<len(cell)*len(cell[0]-1)):
    x,y=distance(i,j, cell, rotations)
    near=np.r_[near,[x,y]]
  else:
    near=np.r_[near,[i,j]]
  return near

# Function forwardHorizontal: to check if the participant can move forward when the rod is
# horizontally orientated.
def forwardHorizontal(i,j,cell):
  r=len(cell)
  c=len(cell[0])
  if (cell[i][j+1]<c*r-1) and (cell[i][j+2]<c*r-1) and (cell[i][j+3]<c*r-1):
    possible_movement=True
  else:
    possible_movement=False
  return possible_movement

# Function forwardVertical: to check if the participant can move forward when the rod is
# vertically orientated.
def forwardVertical(i,j,cell):
  r=len(cell)
  c=len(cell[0])
  if (cell[i][j+1]<c*r-1) and (cell[i+1][j+1]<c*r-1) and (cell[i+2][j+1]<c*r-1):
    possible_movement=True
  else:
    possible_movement=False
  return possible_movement

# Function downwardsHorizontal: to check if the participant can move downwards when the rod is
# horizontally orientated.
def downwardsHorizontal(i,j,cell):
  r=len(cell)
  c=len(cell[0])
  if (cell[i+1][j]<c*r-1) and (cell[i+1][j+1]<c*r-1) and (cell[i+1][j+2]<c*r-1):
    possible_movement=True
  else:
    possible_movement=False
  return possible_movement

# Function downwardsVertical: to check if the participant can move downwards when the rod is
# vertically orientated.
def downwardsVertical(i,j,cell):
  r=len(cell)
  c=len(cell[0])
  if (cell[i+1][j]<c*r-1) and (cell[i+2][j]<c*r-1) and (cell[i+3][j]<c*r-1):
    possible_movement=True
  else:
    possible_movement=False
  return possible_movement

# Function CheckForward: to check if the participant can move forward when the rod is
# either vertically or horizontally orientated.
def CheckForward(rotations,i,j,cell):
  if rotations%2==0:
    fow=forwardHorizontal(i,j,cell)
  else:
    fow=forwardVertical(i,j,cell)
  return fow

# Function CheckDownward: to check if the participant can move downward when the rod is
# either vertically or horizontally orientated.
def CheckDownwards(rotations,i,j,cell):
  if rotations%2==0:
    down=downwardsHorizontal(i,j,cell)
  else:
    down=downwardsVertical(i,j,cell)
  return down

#Function PossibleRotation: to check whether the rod can be rotated.
def PossibleRotation(rotations,i,j,cell):
  r=len(cell)
  c=len(cell[0])
  if (rotations%2==0 and (cell[i][j]!=c*r) and (cell[i-1][j]!=c*r) and (cell[i-1][j+1]!=c*r) and (cell[i-1][j+1]!=c*r)
  and (cell[i][j+1]!=c*r) and (cell[i][j+2]!=c*r) and (cell[i+1][j]!=c*r) and (cell[i+1][j+1]!=c*r) and (cell[i][j+2]!=c*r)):
    rotat=True
  elif (rotations%2!=0 and (cell[i][j]!=c*r) and (cell[i][j-1]!=c*r) and (cell[i+1][j-1]!=c*r) and (cell[i+2][j-1]!=c*r)
  and (cell[i+1][j]!=c*r) and (cell[i+2][j]!=c*r) and (cell[i][j+1]!=c*r) and (cell[i+1][j+1]!=c*r) and (cell[i+2][j+1]!=c*r)):
    rotat=True
  else:
    rotat=False
  return rotat

#Once all the functions are defined, let's begin with the main program:
#Writing down the input and getting the number of columns and rows
labyrinth=[['.','.','.','.','.','.','.','.','.','.'],
           ['.','#','.','.','.','.','#','.','.','.'],
           ['.','#','.','.','.','.','.','.','.','.'],
           ['.','.','.','.','.','.','.','.','.','.'],
           ['.','.','.','.','.','.','.','.','.','.'],
           ['.','#','.','.','.','.','.','.','.','.'],
           ['.','#','.','.','.','#','.','.','.','.'],
           ['.','.','.','.','.','.','#','.','.','.'],
           ['.','.','.','.','.','.','.','.','.','.'],
           ['.','.','.','.','.','.','.','.','.','.']]
rows=len(labyrinth)
columns=len(labyrinth[0])

#Now we create a grid in which every cell contains the number of movements needed to go from the initial
#position to that specific cell, and assigning a huge number to the blocked cells.
cell=np.empty(shape=(rows,columns))
i=0
while i < rows:
  j=0
  while j < columns:
    if labyrinth[i][j]=='.':
       cell[i][j]=i+j
    else:
      cell[i][j]=rows*columns
    j+=1
  i+=1

#Now we define some variables we'll need as the number of rotations, i and j to identify
#the current cell at any time, a boolean variable which remains as False until the goal is
#reached, k to control the number of iterations and an empty arrays (next_cells) to save
#the information of every step we make.
rotations=0
goal=False
i=0
j=0
k=0
next_cells=np.array([])
next_cells=np.r_[next_cells,[0,0]]

#Bool to find the shortest path: it will keep going until we reach the goal
#or until there are too many iterations.
while goal==False and k<10000:
  #First check if we're in the goal already
  if (rotations%2==0 and i==rows-1 and j==columns-3) or (rotations%2!=0 and i==rows-3 and j==columns-1):
    goal=True
  #Now we study the boundaries of the grid to avoid range of indices problems.
  #First check if we're at the bottom of the grid where we can only move forward
  elif (i>=(rows-1) and rotations%2==0) or (i>=(rows-3) and rotations%2!=0):
    fow=CheckForward(rotations,i,j,cell)
    if fow==True:
      #We move forward if we can
      next_cells=np.r_[next_cells,[i,j+1]]
      j+=1
    else:
      #If we can not, we check if we can rotate
      rot=PossibleRotation(rotations,i,j,cell)
      if rot==True:
        #If we can rotate we do so (taking into account the actual orientaton of the rod)
        if rotations%2==0:
          i-=1
          j+=1
        else:
          i+=1
          j-=1
        rotations+=1
        next_cells= np.r_[next_cells,[i,j]]
      else:
        #If not, we change the value of that cell with a huge number and we take a step back
        cell[int(next_cells[len(next_cells)-2])][int(next_cells[len(next_cells)-1])]=rows*columns-1
        next_cells=np.delete(next_cells,len(next_cells)-1)
        next_cells=np.delete(next_cells,len(next_cells)-1)
        i=int(next_cells[len(next_cells)-2])
        j=int(next_cells[len(next_cells)-1])
  #Following, check if we're at the right boundary of the grid where we can only move downwards
  elif (j>=(columns-3) and rotations%2==0) or (j>=(columns-1) and rotations%2!=0):
    down=CheckDownwards(rotations,i,j,cell)
    if down==True:
      #We move downwards if we can
      next_cells=np.r_[next_cells,[i+1,j]]
      i+=1
    else:
      #If we can not, we check if we can rotate
      rot=PossibleRotation(rotations,i,j,cell)
      if rot==True:
        #If we can rotate we do so (taking into account the actual orientaton of the rod)
        if rotations%2==0:
          i-=1
          j+=1
        else:
          i+=1
          j-=1
        rotations+=1
        next_cells= np.r_[next_cells,[i,j]]
      else:
        #If not, we change the value of that cell with a huge number and we take a step back
        cell[int(next_cells[len(next_cells)-2])][int(next_cells[len(next_cells)-1])]=rows*columns-1
        next_cells=np.delete(next_cells,len(next_cells)-1)
        next_cells=np.delete(next_cells,len(next_cells)-1)
        i=int(next_cells[len(next_cells)-2])
        j=int(next_cells[len(next_cells)-1])

  #The following procedure must be followed if we're neither in the goal nor  boundaries of the grid
  else:
    #We first choose where to move using the nearest function
    next_cells= np.r_[next_cells,[nearest(i,j,cell, rotations)[0],nearest(i,j, cell, rotations)[1]]]
    #If the cell where we have to move is fowards we'll follow the following procedure: check if we can
    #move; if we can, we do so and if we can not, we change the value of that cell with a huge number
    #and we take a step back
    if (next_cells[len(next_cells)-2]==i) and (next_cells[len(next_cells)-1]==j+1):
      fow=CheckForward(rotations,i,j,cell)
      if fow==True:
        i=int(next_cells[len(next_cells)-2])
        j=int(next_cells[len(next_cells)-1])
      else:
        cell[int(next_cells[len(next_cells)-2])][int(next_cells[len(next_cells)-1])]=rows*columns-1
        next_cells=np.delete(next_cells,len(next_cells)-1)
        next_cells=np.delete(next_cells,len(next_cells)-1)
        i=int(next_cells[len(next_cells)-2])
        j=int(next_cells[len(next_cells)-1])
    #If the cell where we have to move is downwards we'll follow the following procedure: check if we can
    #move; if we can, we do so and if we can not, we change the value of that cell with a huge number
    #and we take a step back
    elif (next_cells[len(next_cells)-2]==i+1) and (next_cells[len(next_cells)-1]==j):
      down=CheckDownwards(rotations,i,j,cell)
      if down==True:
        i=int(next_cells[len(next_cells)-2])
        j=int(next_cells[len(next_cells)-1])
      else:
        cell[int(next_cells[len(next_cells)-2])][int(next_cells[len(next_cells)-1])]=rows*columns-1
        next_cells=np.delete(next_cells,len(next_cells)-1)
        next_cells=np.delete(next_cells,len(next_cells)-1)
        i=int(next_cells[len(next_cells)-2])
        j=int(next_cells[len(next_cells)-1])
    #If we happen to be in a cell with no possible path
    else:
      #We fist check if we can rotate, doing so if its possible
      rot=PossibleRotation(rotations,i,j,cell)
      if rot==True:
        if rotations%2==0:
          i-=i
          j+=1
        else:
          i+=i
          j-=j
        rotations+=1
        next_cells= np.r_[next_cells,[i,j]]
      else:
        #If it's not possible,we change the value of that cell with a huge number and we take a step back
        cell[int(next_cells[len(next_cells)-2])][int(next_cells[len(next_cells)-1])]=rows*columns-1
        next_cells=np.delete(next_cells,len(next_cells)-1)
        next_cells=np.delete(next_cells,len(next_cells)-1)
        i=int(next_cells[len(next_cells)-2])
        j=int(next_cells[len(next_cells)-1])

  #Now we just have to make sure we're not at any dead ends for both different orientations,
  #and if we are, we take a step back. In case we're back at the beggining, we end the loop
  #because it means the labyrinth has no possible solution.
  if rotations%2==0 and (j<columns-3 or i< rows-1):
    if (cell[i+1][j]>=rows*columns-1) and (cell[i][j+1]>=rows*columns-1):
      if i==0 and j==0:
        k=10000
      else:
        next_cells=np.delete(next_cells,len(next_cells)-1)
        next_cells=np.delete(next_cells,len(next_cells)-1)
  if rotations%2!=0 and (j<columns-1 or i< rows-3):
    if (cell[i+1][j]>=rows*columns-1) and (cell[i][j+1]>=rows*columns-1):
      if i==0 and j==0:
        k=10000
      else:
        next_cells=np.delete(next_cells,len(next_cells)-1)
        next_cells=np.delete(next_cells,len(next_cells)-1)
  #We sum every iteration
  k+=1

#To print the result: if the number of iterations is above the limit
# it means the labrinth has no possible solution. Otherwise, the number of
# movements corresponds to the lenght of the array where we saved every
# step divided by two as we have two coordinates for each cell.
if k<10000:
  print(int((len(next_cells)-1)/2))
else:
  print(-1)


16
